In [ ]:
# Auto reload local files
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
# Make files in src/ available to notebook
import sys
if 'src' not in sys.path:
    sys.path.insert(0, 'src')

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge

from technical_signals import TechnicalSignals, percent_change
import datastore as ds
import backtest as bt
from strategy import TechnicalIndicatorsModelStrategy, DummyStrategy

# XXX NOTE VERY IMPORTANT
# THIS FILTERS ALL WARNINGS - ta library is giving annoying warnings.
# REMOVE AND RESTART before modifying any bt code
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 10]

spy_constituents = list(pd.read_csv('../../data/spy_constituents.csv', header=0)['Symbol'])
random.shuffle(spy_constituents)

with open('../../data/cryptocurrencies_list.json') as f:
    cryptocurrencies = json.loads(f.read())['top_800_coinmarketcap']
    random.shuffle(cryptocurrencies)
    cryptocurrencies = list(map(lambda cur: f"{cur}-USD", cryptocurrencies))

tickers = cryptocurrencies
start_date = "2000-01-01"
end_date = "2027-01-01"
predict_window = 7
train_test_ratio = 0.1

# Build technical signals
#ds.download_daily_candlesticks(tickers, start_date, end_date)
candles = ds.get_daily_candlesticks(tickers, start_date, end_date)
tsigs = None
for ticker in tickers:
    try:
        df = candles[ticker]
        df = df[df.index < int(len(df.index)*train_test_ratio)]
        if tsigs is None:
            tsigs = TechnicalSignals(df, predict_window)
        else:
            tsigs = tsigs.append(TechnicalSignals(df, predict_window))
    except Exception as ex:
        print(f"Failed to add {ticker} to technical signals")
        print(ex)

strategies = [
    ('MLP', TechnicalIndicatorsModelStrategy(MLPRegressor(), window=7, cutoff=0.6, bias=0.1, pretrain_tsigs=tsigs)),
    ('Random Forest', TechnicalIndicatorsModelStrategy(RandomForestRegressor(n_jobs=-1, criterion='absolute_error'), window=14, cutoff=0.6, bias=0.2, pretrain_tsigs=tsigs)),
    ('SGD', TechnicalIndicatorsModelStrategy(SGDRegressor(), window=7, cutoff=0.5, bias=0.1, pretrain_tsigs=tsigs)),
    ('SVR', TechnicalIndicatorsModelStrategy(SVR(), window=7, cutoff=0.5, bias=0.1, pretrain_tsigs=tsigs)),
    ('Dummy', DummyStrategy())
]

"""
for name, strategy in strategies:
    print(f"========== TESTING MODEL: {name} ===========\n\n")
    bt_summary = bt.comprehensive_backtest(strategy, tickers, plot=True, skip_index=252*15, processes=8)
    print(bt_summary)
"""


for name, strategy in strategies:
    print(f"========== TESTING MODEL: {name} ===========\n\n")
    bt_summary = bt.comprehensive_backtest(strategy, tickers, start_date, end_date, plot=True, train_test_ratio=train_test_ratio, processes=8)
    print(bt_summary)

Failed to add OP-USD to technical signals
index 119 is out of bounds for axis 0 with size 76
Failed to add ALPINE-USD to technical signals
index 119 is out of bounds for axis 0 with size 86
Failed to add BTTOLD-USD to technical signals
'date'
Failed to add SHIRYO-USD to technical signals
'date'
Failed to add MEV-USD to technical signals
index 119 is out of bounds for axis 0 with size 70
Failed to add QOM-USD to technical signals
index 119 is out of bounds for axis 0 with size 74
Failed to add USDD-USD to technical signals
index 59 is out of bounds for axis 0 with size 51
Failed to add CULT-USD to technical signals
index 119 is out of bounds for axis 0 with size 95
Failed to add SOS-USD to technical signals
'date'
Failed to add USTC-USD to technical signals
'date'
Failed to add LEVER-USD to technical signals
index 59 is out of bounds for axis 0 with size 15
Failed to add DFG-USD to technical signals
index 119 is out of bounds for axis 0 with size 68
Failed to add WVLX-USD to technical s